In [2]:

# Step 1: Imports
import os
import glob
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [3]:
# Step 2: Load image paths and labels
base_path = "Cheat Database"
scenario_dirs = sorted([os.path.join(base_path, d) for d in os.listdir(base_path) if d.startswith("Scenario")])

image_paths = []
labels = []

for scenario_path in scenario_dirs:
    label = int(scenario_path.split("Scenario ")[1]) - 1
    for img_path in glob.glob(os.path.join(scenario_path, "*.jpg")):
        image_paths.append(img_path)
        labels.append(label)


In [4]:
# Step 3: Create DataFrame
df = pd.DataFrame({"image_path": image_paths, "label": labels})
print(df.head())

                                  image_path  label
0  Cheat Database/Scenario 1/S1_00000002.jpg      0
1  Cheat Database/Scenario 1/S1_00000001.jpg      0
2  Cheat Database/Scenario 1/S1_00000000.jpg      0
3  Cheat Database/Scenario 1/S1_00000004.jpg      0
4  Cheat Database/Scenario 1/S1_00000003.jpg      0


In [5]:
# Step 4: Load and preprocess images
X = []
y = []

for idx, row in df.iterrows():
    img = cv2.imread(row["image_path"])
    if img is None:
        continue
    img = cv2.resize(img, (224, 224))
    X.append(img)
    y.append(row["label"])

X = np.array(X) / 255.0
y = np.array(y)
y_cat = to_categorical(y)


In [1]:
# Step 5: Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

NameError: name 'train_test_split' is not defined

In [ ]:
# Step 6: Define model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(len(np.unique(y)), activation="softmax")
])

In [ ]:
# Step 7: Compile and train
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# Step 8: Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")